In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
from scipy import signal
from scipy.signal import find_peaks
from scipy.integrate import cumtrapz
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# 変数の宣言
sampling_hz = 100

In [ ]:
# csvファイルの読み込み
file = "up_down_filter"
csv_Accelerometer = f'data/{file}/Accelerometer.csv'
csv_Gyroscope = f'data/{file}/Gyroscope.csv'
csv_Pressure = f'data/{file}/Pressure.csv'
raw_Accelerometer = pd.read_csv(csv_Accelerometer)
raw_Gyroscope = pd.read_csv(csv_Gyroscope)
raw_Pressure = pd.read_csv(csv_Pressure)

print(len(raw_Accelerometer))
print(len(raw_Gyroscope))
print(len(raw_Pressure))


In [ ]:
df = pd.read_csv(csv_Accelerometer)
# normを計算
df['norm'] = (df['x']**2 + df['y']**2 + df['z']**2)**0.5
# 移動平均を計算
df['low_norm'] = df['norm'].rolling(window=sampling_hz).mean()
# df['low_norm'] = df['low_norm'].rolling(window=10).mean()
fig = plt.figure(figsize=(15, 4))
plt.title(f'歩数推定', fontsize=20)
plt.plot(df['t'], df['low_norm'], zorder=1)
plt.xlabel('時間(s)')
plt.ylabel('ノルム(m/s^2)')
# 赤点を描画
peek, _ = signal.find_peaks(df['low_norm'],distance=40, height=10)
plt.scatter(df['t'][peek], df['low_norm']
            [peek], s=30, color='red', zorder=2)
plt.xlim(20, 90)
plt.show()    

In [ ]:
# Constants
DT = 1 / sampling_hz

angle_data = np.cumsum(raw_Gyroscope * DT) *-1

# Plot angle data
fig = plt.figure(figsize=(5, 4))
plt.title('角度')
plt.xlabel('時間 (s)')
plt.ylabel('角度 x(rad)')
plt.plot(angle_data['x'],label='x')
plt.plot(angle_data['y'],label='y')
plt.plot(angle_data['z'],label='z')
plt.legend()
plt.show()


# gyro_norm = (gyro_data['x']**2)*0.5
# gyro_norm = np.cumsum(gyro_norm * DT)

# # Plot angle data
# plt.xlabel('時間 (s)')
# plt.ylabel('角度 x(rad)')
# plt.plot(gyro_norm)
# plt.show()

In [ ]:
# csvファイルの読み込み
file = "walk"
csv_Accelerometer = f'data/{file}/Accelerometer.csv'
csv_Gyroscope = f'data/{file}/Gyroscope.csv'
csv_Pressure = f'data/{file}/Pressure.csv'
raw_Accelerometer = pd.read_csv(csv_Accelerometer)
raw_Gyroscope = pd.read_csv(csv_Gyroscope)
raw_Pressure = pd.read_csv(csv_Pressure)

DT = 1 / sampling_hz 


def search_peek_time(file_path):
    df = pd.read_csv(file_path)
    df['norm'] = (df['x']**2 + df['y']**2 + df['z']**2)**0.5
    df['low_norm'] = df['norm'].rolling(window=10).mean()
    # ピークを検出
    peek, _ = signal.find_peaks(df['low_norm'], height=12)
    ## peekのtを取得
    peek_t = df['t'][peek]

    return np.round(peek_t,2)

gyro_data = pd.read_csv(csv_Gyroscope)
# 't'列を除外したデータを取得
gyro_data_no_t = gyro_data.drop(columns=['t'])
# 'x', 'y', 'z'列のみを積分
angle_data_no_t = np.cumsum(gyro_data_no_t) * DT
# 積分したデータに't'列を結合
angle_data = pd.concat([gyro_data['t'], angle_data_no_t], axis=1)
angle_data['rounded_t'] = np.round(angle_data['t'], 2)
# search_peek_time()のkeysを取得
peek_t_values = search_peek_time(csv_Accelerometer).values
angle_data_peek = angle_data[angle_data['rounded_t'].isin(peek_t_values)]
angle_data_peek_x = angle_data_peek['x']
print(angle_data_peek)

print(angle_data_peek_x)

plt.title('角度 x')
plt.xlabel('時間 (s)')
plt.ylabel('角度 x(rad)')
plt.plot(angle_data['t'], angle_data['x'])
plt.show()

In [ ]:
# csvファイルの読み込み
file = "up_down_filter"
csv_Accelerometer = f'data/{file}/Accelerometer.csv'
csv_Gyroscope = f'data/{file}/Gyroscope.csv'
csv_Pressure = f'data/{file}/Pressure.csv'
raw_Accelerometer = pd.read_csv(csv_Accelerometer)
raw_Gyroscope = pd.read_csv(csv_Gyroscope)
raw_Pressure = pd.read_csv(csv_Pressure)

sampling_hz = 100
DT = 1 / sampling_hz 
STEP_LENGTH = 50  # cm


def search_peek_time(file_path):
    df = pd.read_csv(file_path)
    df['norm'] = (df['x']**2 + df['y']**2 + df['z']**2)**0.5
    df['low_norm'] = df['norm'].rolling(window=sampling_hz).mean()
    # ピークを検出
    peek, _ = signal.find_peaks(df['low_norm'],distance=40, height=10)
    ## peekのtを取得
    peek_t = df['t'][peek]

    return np.round(peek_t,2)

gyro_data = pd.read_csv(csv_Gyroscope)
# 't'列を除外したデータを取得
gyro_data_no_t = gyro_data.drop(columns=['t'])
# 'x', 'y', 'z'列のみを積分
angle_data_no_t = np.cumsum(gyro_data_no_t )* DT
# 積分したデータに't'列を結合
angle_data = pd.concat([gyro_data['t'], angle_data_no_t], axis=1)
angle_data['rounded_t'] = np.round(angle_data['t'], 2)
# search_peek_time()のkeysを取得
peek_t_values = search_peek_time(csv_Accelerometer).values
angle_data_peek = angle_data[angle_data['rounded_t'].isin(peek_t_values)]
angle_data_peek_x = angle_data_peek['x']

x_displacement = STEP_LENGTH * np.cos(angle_data_peek_x)
y_displacement = STEP_LENGTH * np.sin(angle_data_peek_x)
x_cumulative = np.cumsum(x_displacement)
y_cumulative = np.cumsum(y_displacement)*-1
# 初期座標(0,0)が描画されるように挿入
x_cumulative = np.insert(x_cumulative, 0, 0)
y_cumulative = np.insert(y_cumulative, 0, 0)

plt.figure()
plt.plot(x_cumulative, y_cumulative, marker='o')

# X軸とY軸の範囲を揃える

# 軸を揃える
plt.gca().set_aspect('equal', adjustable='box')
#軸を揃える
plt.xlabel('X 方向 (cm)')
plt.ylabel('Y 方向 (cm)')
plt.title('歩行推定')
plt.grid()
plt.show()

In [ ]:
# Constants
ACC_FILE_PATH = './data/Accelerometer.csv'
GYRO_FILE_PATH = './data/Gyroscope.csv'
DT = 1 / sampling_hz 
STEP_LENGTH = 30  # cm


def search_peek_time(file_path):
    df = pd.read_csv(file_path)
    df['norm'] = (df['x']**2 + df['y']**2 + df['z']**2)**0.5
    df['low_norm'] = df['norm'].rolling(window=100).mean()
    # ピークを検出
    peek, _ = signal.find_peaks(df['low_norm'],distance=40, height=10)
    ## peekのtを取得
    peek_t = df['t'][peek]

    return np.round(peek_t,2)

gyro_data = pd.read_csv(GYRO_FILE_PATH)
# 't'列を除外したデータを取得
gyro_data_no_t = gyro_data.drop(columns=['t'])
# 'x', 'y', 'z'列のみを積分
angle_data_no_t = np.cumsum(gyro_data_no_t )* DT
# 積分したデータに't'列を結合
angle_data = pd.concat([gyro_data['t'], angle_data_no_t], axis=1)
angle_data['rounded_t'] = np.round(angle_data['t'], 2)
# search_peek_time()のkeysを取得
peek_t_values = search_peek_time(ACC_FILE_PATH).values
angle_data_peek = angle_data[angle_data['rounded_t'].isin(peek_t_values)]
angle_data_peek_x = angle_data_peek['x']

x_displacement = STEP_LENGTH * np.cos(angle_data_peek_x)
y_displacement = STEP_LENGTH * np.sin(angle_data_peek_x)
x_cumulative = np.cumsum(x_displacement)
y_cumulative = np.cumsum(y_displacement)*-1
# 初期座標(0,0)が描画されるように挿入
x_cumulative = np.insert(x_cumulative, 0, 0)
y_cumulative = np.insert(y_cumulative, 0, 0)

plt.figure()
plt.plot(x_cumulative, y_cumulative, marker='o')

# X軸とY軸の範囲を揃える

# 軸を揃える
plt.gca().set_aspect('equal', adjustable='box')
#軸を揃える
plt.xlabel('X 方向 (cm)')
plt.ylabel('Y 方向 (cm)')
plt.title('歩行推定')
plt.grid()
plt.show()

In [ ]:
df = pd.read_csv(csv_Pressure)
df['p_2'] = df['p'].rolling(window=8).mean()

#描画エリアの作成
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

print(len(x_cumulative))
print(len(y_cumulative))
 
#散布図の作成
ax.scatter(x_cumulative, y_cumulative,df['p'],s=40,marker='o')


# 軸を揃える
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('X 方向 (cm)')
plt.ylabel('Y 方向 (cm)')
plt.title('歩行推定')
#描画
plt.show()